In [1]:
from transformers import pipeline
import pandas as pd

/Users/crazyk/opt/anaconda3/envs/tf-2307/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = '''
Dear Amazon, last week I ordered an Optimus Prime action figure from your
online store in Germany. Unfortunately, when I opened the package, I discovered
to my horror that fI had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the 
issue, I demand an excange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear from
you soon. Sincerely, Bumblebee.
'''

In [3]:
classifier = pipeline('text-classification')
outputs = classifier(text)
df_output = pd.DataFrame(outputs)

print(df_output)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


      label     score
0  NEGATIVE  0.923832


In [4]:
outputs

[{'label': 'NEGATIVE', 'score': 0.923831582069397}]

### <b style="color: #F9A440">개체명 인식</b>

In [5]:
ner_tagger = pipeline('ner', aggregation_strategy='simple')
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,ORG,0.887620,Amazon,6,12
1,MISC,0.992057,Optimus Prime,37,50
2,LOC,0.999746,Germany,91,98
3,MISC,0.609341,Mega,210,214
4,PER,0.518909,##tron,214,218
5,ORG,0.648227,Decept,255,261
6,MISC,0.473363,##icons,261,266
7,MISC,0.672558,Megatron,352,360
8,MISC,0.990489,Optimus Prime,369,382
9,PER,0.824856,Bumblebee,504,513


### <b style="color: #F9A440">질문 답변</b>

In [6]:
reader = pipeline('question-answering')
question = 'What does the customer want?'
outputs = reader(question=question, context=text)
result = pd.DataFrame([outputs])
result

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.54499,338,360,an excange of Megatron


In [7]:
outputs

{'score': 0.5449897646903992,
 'start': 338,
 'end': 360,
 'answer': 'an excange of Megatron'}

### <b style="color: #F9A440">요약</b>

In [9]:
summarizer = pipeline('summarization')
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your min_length=56 must be inferior than your max_length=45.


{'summary_text': ' Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when he opened the package, he discovered that he had been sent an action figure of Megatron instead. As a lifelong'}


In [10]:
outputs

[{'summary_text': ' Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when he opened the package, he discovered that he had been sent an action figure of Megatron instead. As a lifelong'}]

### <b style="color: #F9A440">번역</b>

In [11]:
translator = pipeline(
    'translation_en_to_de',
    model='Helsinki-NLP/opus-mt-en-de'
)
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

/Users/crazyk/opt/anaconda3/envs/tf-2307/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Liebe Amazon, letzte Woche bestellte ich eine Optimus Prime Action Figur von Ihrem Online-Shop in Deutschland. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass fI stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, dass Sie mein Dilemma verstehen können. Um das Problem zu lösen, fordere ich eine Ausrufung von Megatron für die Optimus Prime Figur, die ich bestellt hatte.


### <b style="color: #F9A440">텍스트 생성</b>

In [12]:
generator = pipeline('text-generation')
response = 'Dear Bumblebee, I am sorry to hear that your order was mixed up'
prompt = text + '\n\nCustomer service response:\n' + response
outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Dear Amazon, last week I ordered an Optimus Prime action figure from your
online store in Germany. Unfortunately, when I opened the package, I discovered
to my horror that fI had been sent an action figure of Megatron instead! As a lifelong
enemy of the Decepticons, I hope you can understand my dilemma. To resolve the 
issue, I demand an excange of Megatron for the Optimus Prime figure I ordered.
Enclosed are copies of my records concerning this purchase. I expect to hear from
you soon. Sincerely, Bumblebee.


Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up, but this case is

meant for no issue. My understanding is that the first question was addressed to you by your

store, whereas this particular issue of orders is simply not relevant to the current

model. Please confirm that you are not dealing with a different machine


<module 'sentencepiece' from '/Users/crazyk/opt/anaconda3/envs/tf-2307/lib/python3.8/site-packages/sentencepiece/__init__.py'>